In [0]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = (
    SparkSession.builder
    .config('spark.ui.port', '0')
    .config(
        "spark.sql.warehouse.dir",
        f"/user/{username}/warehouse"
    )
    .enableHiveSupport()
    .getOrCreate()
)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
from pyspark.sql.functions import *


In [0]:
spark = SparkSession.builder.getOrCreate()


In [0]:
data = [Row(1, "John", 30, "Sales", 50000.0),
 Row(2, "Alice", 28, "Marketing", 60000.0),
 Row(3, "Bob", 32, "Finance", 55000.0),
 Row(4, "Sarah", 29, "Sales", 52000.0),
 Row(5, "Mike", 31, "Finance", 58000.0),
 Row(6, "Jane", 35, "Marketing", 62000.0),
 Row(7, "Tom", 33, "Sales", 54000.0),
 Row(8, "Lucy", 27, "Marketing", 59000.0),
 Row(9, "David", 36, "Finance", 65000.0),
 Row(10, "Emma", 26, "Marketing", 61000.0)
]

In [0]:
data

[<Row(1, 'John', 30, 'Sales', 50000.0)>,
 <Row(2, 'Alice', 28, 'Marketing', 60000.0)>,
 <Row(3, 'Bob', 32, 'Finance', 55000.0)>,
 <Row(4, 'Sarah', 29, 'Sales', 52000.0)>,
 <Row(5, 'Mike', 31, 'Finance', 58000.0)>,
 <Row(6, 'Jane', 35, 'Marketing', 62000.0)>,
 <Row(7, 'Tom', 33, 'Sales', 54000.0)>,
 <Row(8, 'Lucy', 27, 'Marketing', 59000.0)>,
 <Row(9, 'David', 36, 'Finance', 65000.0)>,
 <Row(10, 'Emma', 26, 'Marketing', 61000.0)>]

In [0]:
schema = StructType([
 StructField("id", IntegerType(), nullable=False),
 StructField("name", StringType(), nullable=False),
 StructField("age", IntegerType(), nullable=False),
 StructField("department", StringType(), nullable=False),
 StructField("salary", DoubleType(), nullable=False)
])

In [0]:
schema

StructType([StructField('id', IntegerType(), False), StructField('name', StringType(), False), StructField('age', IntegerType(), False), StructField('department', StringType(), False), StructField('salary', DoubleType(), False)])

In [0]:
employeeDF = spark.createDataFrame(data, schema)

In [0]:
employeeDF.show()


+---+-----+---+----------+-------+
| id| name|age|department| salary|
+---+-----+---+----------+-------+
|  1| John| 30|     Sales|50000.0|
|  2|Alice| 28| Marketing|60000.0|
|  3|  Bob| 32|   Finance|55000.0|
|  4|Sarah| 29|     Sales|52000.0|
|  5| Mike| 31|   Finance|58000.0|
|  6| Jane| 35| Marketing|62000.0|
|  7|  Tom| 33|     Sales|54000.0|
|  8| Lucy| 27| Marketing|59000.0|
|  9|David| 36|   Finance|65000.0|
| 10| Emma| 26| Marketing|61000.0|
+---+-----+---+----------+-------+



In [0]:
employeeDF.printSchema()
employeeDF.select("name", "salary").show()

root
 |-- id: integer (nullable = false)
 |-- name: string (nullable = false)
 |-- age: integer (nullable = false)
 |-- department: string (nullable = false)
 |-- salary: double (nullable = false)

+-----+-------+
| name| salary|
+-----+-------+
| John|50000.0|
|Alice|60000.0|
|  Bob|55000.0|
|Sarah|52000.0|
| Mike|58000.0|
| Jane|62000.0|
|  Tom|54000.0|
| Lucy|59000.0|
|David|65000.0|
| Emma|61000.0|
+-----+-------+



In [0]:
#calculate the average salary for each department
averagesalbydept = employeeDF.groupBy("department").agg(avg("salary").alias("avg_salary"))
averagesalbydept.show()

+----------+------------------+
|department|        avg_salary|
+----------+------------------+
|     Sales|           52000.0|
| Marketing|           60500.0|
|   Finance|59333.333333333336|
+----------+------------------+



In [0]:
from pyspark.sql.functions import avg

averagesalbydept = (
    employeeDF.groupBy("department")
    .agg(
        avg("salary").alias("avg_salary")
    )
)
display(averagesalbydept)

department,avg_salary
Sales,52000.0
Marketing,60500.0
Finance,59333.333333333336


In [0]:
#Add a new column named "bonus" that is 10% of the salary for all employees
employeeDF = employeeDF.selectExpr('*', "salary * 0.1 as bonus")
employeeDF.show()

+---+-----+---+----------+-------+------+
| id| name|age|department| salary| bonus|
+---+-----+---+----------+-------+------+
|  1| John| 30|     Sales|50000.0|5000.0|
|  2|Alice| 28| Marketing|60000.0|6000.0|
|  3|  Bob| 32|   Finance|55000.0|5500.0|
|  4|Sarah| 29|     Sales|52000.0|5200.0|
|  5| Mike| 31|   Finance|58000.0|5800.0|
|  6| Jane| 35| Marketing|62000.0|6200.0|
|  7|  Tom| 33|     Sales|54000.0|5400.0|
|  8| Lucy| 27| Marketing|59000.0|5900.0|
|  9|David| 36|   Finance|65000.0|6500.0|
| 10| Emma| 26| Marketing|61000.0|6100.0|
+---+-----+---+----------+-------+------+



In [0]:
# Group the data by department and find the employee with the highest salary in each department
from pyspark.sql.window import window
from pyspark.sql.functions import max, row_number

employeeDF = employeeDF.withColumn("rn", row_number()